In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
filepath = "./data/1585 Ga raw.xlsx"

In [ ]:
DODI_C_20 = pd.read_excel(filepath, sheet_name = 0,engine = "openpyxl", header = None)
PROD_C_20 = pd.read_excel(filepath, sheet_name = 1,engine = "openpyxl", header = None)
DODI_D_20 = pd.read_excel(filepath, sheet_name = 2,engine = "openpyxl", header = None)
PROD_D_20 = pd.read_excel(filepath, sheet_name = 3,engine = "openpyxl", header = None)
DODI_C_rt = pd.read_excel(filepath, sheet_name = 4,engine = "openpyxl", header = None)
PROD_C_rt = pd.read_excel(filepath, sheet_name = 5,engine = "openpyxl", header = None)
DODI_D_rt = pd.read_excel(filepath, sheet_name = 6,engine = "openpyxl", header = None)
PROD_D_rt = pd.read_excel(filepath, sheet_name = 7,engine = "openpyxl", header = None)
DODI_C_60 = pd.read_excel(filepath, sheet_name = 8,engine = "openpyxl", header = None)
PROD_C_60 = pd.read_excel(filepath, sheet_name = 9,engine = "openpyxl", header = None)
DODI_D_60 = pd.read_excel(filepath, sheet_name = 10,engine = "openpyxl", header = None)
PROD_D_60 = pd.read_excel(filepath, sheet_name = 11,engine = "openpyxl", header = None)

batteryListName= [
    ['DODI_C_20','DODI_C_rt','DODI_C_60'],
    ['DODI_D_20','DODI_D_rt','DODI_D_60'],
    ['PROD_C_20','PROD_C_rt','PROD_C_60'],
    ['PROD_D_20','PROD_D_rt','PROD_D_60']
]

batteryList = [
    [DODI_C_20, DODI_C_rt, DODI_C_60],
    [DODI_D_20,DODI_D_rt,DODI_D_60],
    [PROD_C_20,PROD_C_rt,PROD_C_60],
    [PROD_D_20,PROD_D_rt,PROD_D_60]
]

for i in range(0,4):
    for j in range(0,3):
        dataSet = batteryList[i][j]
        dataSet.replace('', np.nan, inplace=True)

In [ ]:
def printBatteryData():
    count = 1
    plt.figure(figsize=(16,16))
    for i in range(0,4):
        for j in range(0,3):
            dataSet = batteryList[i][j]
            plt.subplot(4,3,count)
            plt.title(batteryListName[i][j])
            for k in range(1, (int)(dataSet.shape[1])-1, 2):
                plt.plot(dataSet.iloc[:,[k+1]], dataSet.iloc[:,[k]])
            plt.xlabel('coulomb')
            plt.ylabel('Volatage')
            count += 1

In [ ]:
printBatteryData()

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv3D, Conv1D
from sklearn.model_selection import KFold

In [ ]:
kfold_num = 5
window_size = 20
horizon = 1
filters = 512
kernel_size = 3
dimension = 3
epochs = 30
batch_size = 50

In [ ]:
class DataPreprocessing:
    def getSequence(dataFrame, temp, i):
        seq = dataFrame.iloc[:,[2*i,2*i+1]].copy()
        seq.dropna(axis=0, inplace=True)
        seq = seq.to_numpy()
        tempSeq = np.array(seq.shape[0], dtype = float)
        tempSeq = np.full((seq.shape[0],1),temp)
        seq = np.append(seq, tempSeq[:,-1:], axis = 1)
        return seq
    #시계열 데이터를 윈도우 단위로 자르는 함수
    def seq2dataset(seq,window,horizon):
        X=[]; Y=[]
        for i in range(len(seq)-(window+horizon)+1):
            x=seq[i:(i+window)]
            y=(seq[i+window+horizon-1])
            X.append(x); Y.append(y)
        return np.array(X), np.array(Y)

In [ ]:
plt.title("asdf")
df = DODI_C_60.iloc[:,[0,1]].copy()
plt.plot(df.iloc[:,[0]], df.iloc[:,[1]])
plt.xlabel('coulomb')
plt.ylabel('Volatage')

In [ ]:
class DrawPlot:
    def drawValidCurve(hist):
        plt.plot(hist.history['mae'])
        plt.plot(hist.history['val_mae'])
        plt.title('Model mae')
        plt.ylabel('mae')
        plt.xlabel('Epoch')
        plt.legend(['Train','Validation'], loc='best')
        plt.grid()
        plt.show()
    
    def drawPrediction(i,j,pred,y_test):
        plt.figure(figsize=(8,8))
        x_range=range(y_test.shape[0])
        plt.plot(x_range,y_test[x_range,1],color='red',linestyle='dotted')
        plt.plot(x_range,pred[x_range,1],color='green',linestyle='dashed')
        plt.title(batteryListName[i][j])
        plt.legend(['True value','Predicted value'], loc='best')
        plt.grid()
        plt.show()
        

In [ ]:
def deepLearn(dataList):
    for num1 in range(0,2):
        for num2 in range(0,3):
            if num2==0:
                temp = -20
            elif num2==1:
                temp = 20
            else:
                temp = 60
            dataFrame = dataList[num1][num2]
            len = (int)(dataFrame.shape[1]/2)
            for i in range(len):
                seq = DataPreprocessing.getSequence(dataFrame, temp, i)
                
                kfold = KFold(n_splits=kfold_num)
                
                X,Y = DataPreprocessing.seq2dataset(seq,window_size,horizon)
                for train_idx, test_idx in kfold.split(X):
                    x_train, x_test = X[train_idx],X[test_idx]
                    y_train, y_test = Y[train_idx],Y[test_idx]

                    model = Sequential()
                    model.add(Conv1D(filters = filters, kernel_size = kernel_size, input_shape = (window_size, dimension),padding = 'same', activation = 'relu'))
                    model.add(Dropout(0.2))
                    model.add(Conv1D(filters = filters, kernel_size = kernel_size, padding = 'same', activation = 'relu'))
                    model.add(Dropout(0.2))
                    model.add(Conv1D(filters = filters, kernel_size = kernel_size, padding = 'same', activation = 'relu'))
                    model.add(Dropout(0.2))
                    model.add(LSTM(units=64,input_shape=x_train[0].shape, return_sequences=True))
                    model.add(LSTM(units=64, return_sequences=False))
                    model.add(Dense(3))
                    model.compile(loss='mae')
                    model.compile(loss='mae', optimizer='adam', metrics='mae')
                    hist=model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size,validation_data=(x_test,y_test),verbose=1)

                    ev=model.evaluate(x_test,y_test,verbose=1)
                    print("손실 함수:", ev[0], "MAE:", ev[1])

                    pred=model.predict(x_test)
                    
                    # division = x_test.shape[0]
                    print("LSTM 평균절댓값백분율오차(MAPE):",sum(abs(y_test-pred)/y_test)/x_test.shape[0])

                    DrawPlot.drawValidCurve(hist)

                    pred = model.predict(X)
                    y_test = Y
                    
                    DrawPlot.drawPrediction(num1, num2, pred, y_test)

                    model.save('./ohs_model')
            

In [ ]:
deepLearn(batteryList)